### Spark Moive Recommendation
In this notebook, we will use an Alternating Least Squares (ALS) algorithm with Spark APIs to predict the ratings for the movies in [MovieLens small dataset](https://grouplens.org/datasets/movielens/latest/)

To recommend movies to a new user, we can let him / her choose speific interesting genres first, and then give him / her movies recommendation in those genres.

In [3]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from pyspark.sql import functions as F
from pyspark.sql.types import IntegerType, FloatType
import math
dbutils.library.installPyPI("koalas")
import databricks.koalas as ks
from numpy import dot
from numpy.linalg import norm
%matplotlib inline

PyPI package koalas has been installed already. The previously-installed package is `koalas`. To resolve this issue detach and re-attach the notebook to create a new environment or rename the package.

In [4]:
import os
os.environ["PYSPARK_PYTHON"] = "python3"

## Part1: Data ETL and Data Exploration

In [6]:
from pyspark.sql import SparkSession
spark = SparkSession \
    .builder \
    .appName("moive analysis") \
    .config("spark.some.config.option", "some-value") \
    .getOrCreate()

In [7]:
movies_df = spark.read.load("/FileStore/tables/movies.csv", format='csv', header = True)
ratings_df = spark.read.load("/FileStore/tables/ratings.csv", format='csv', header = True)
links_df = spark.read.load("/FileStore/tables/links.csv", format='csv', header = True)
tags_df = spark.read.load("/FileStore/tables/tags.csv", format='csv', header = True)

In [8]:
movies_df.show(5)

+-------+--------------------+--------------------+
movieId| title| genres|
+-------+--------------------+--------------------+
 1| Toy Story (1995)|Adventure|Animati...|
 2| Jumanji (1995)|Adventure|Childre...|
 3|Grumpier Old Men ...| Comedy|Romance|
 4|Waiting to Exhale...|Comedy|Drama|Romance|
 5|Father of the Bri...| Comedy|
+-------+--------------------+--------------------+
only showing top 5 rows

In [9]:
ratings_df.show(5)

+------+-------+------+---------+
userId|movieId|rating|timestamp|
+------+-------+------+---------+
 1| 1| 4.0|964982703|
 1| 3| 4.0|964981247|
 1| 6| 4.0|964982224|
 1| 47| 5.0|964983815|
 1| 50| 5.0|964982931|
+------+-------+------+---------+
only showing top 5 rows

In [10]:
links_df.show(5)

+-------+-------+------+
movieId| imdbId|tmdbId|
+-------+-------+------+
 1|0114709| 862|
 2|0113497| 8844|
 3|0113228| 15602|
 4|0114885| 31357|
 5|0113041| 11862|
+-------+-------+------+
only showing top 5 rows

In [11]:
tags_df.show(5)

+------+-------+---------------+----------+
userId|movieId| tag| timestamp|
+------+-------+---------------+----------+
 2| 60756| funny|1445714994|
 2| 60756|Highly quotable|1445714996|
 2| 60756| will ferrell|1445714992|
 2| 89774| Boxing story|1445715207|
 2| 89774| MMA|1445715200|
+------+-------+---------------+----------+
only showing top 5 rows

In [12]:
print("If there is missing data in each table?")
print('movie_df: {}'.format(movies_df.count() == movies_df.na.drop().count()))
print('ratings_df: {}'.format(ratings_df.count() == ratings_df.na.drop().count()))
print('links_df: {}'.format(links_df.count() == links_df.na.drop().count()))
print('tags_df: {}'.format(tags_df.count() == tags_df.na.drop().count()))

If there is missing data in each table?
movie_df: True
ratings_df: True
links_df: False
tags_df: True

In [13]:
tmp1 = ratings_df.groupBy("userID").count().toPandas()['count'].min()
tmp2 = ratings_df.groupBy("movieId").count().toPandas()['count'].min()
print('For the users that rated movies and the movies that were rated:')
print('Minimum number of ratings per user is {}'.format(tmp1))
print('Minimum number of ratings per movie is {}'.format(tmp2))

For the users that rated movies and the movies that were rated:
Minimum number of ratings per user is 20
Minimum number of ratings per movie is 1

In [14]:
tmp1 = sum(ratings_df.groupBy("movieId").count().toPandas()['count'] == 1)
tmp2 = ratings_df.select('movieId').distinct().count()
print('{} out of {} movies are rated by only one user'.format(tmp1, tmp2))

3446 out of 9724 movies are rated by only one user

In [15]:
movies_df.registerTempTable("movies")
ratings_df.registerTempTable("ratings")
links_df.registerTempTable("links")
tags_df.registerTempTable("tags")

### Q1: The number of Users

In [17]:
user_count=ratings_df.select('userID').distinct().count()
print('The total number of distinct user is: {}'.format(user_count))

The total number of distinct user is: 610

### Q2: The number of Movies

In [19]:
movie_count=movies_df.select('movieID').distinct().count()
print('The total number of distinct movie is: {}'.format(movie_count))

The total number of distinct movie is: 9742

### Q3:  How many movies are rated by users? List movies not rated before

In [21]:
# numbers of movies that are rated by users
rated = ratings_df.select('movieID')\
.where('rating is not null')\
.withColumnRenamed('movieId', 'ratedId').distinct()
rated_count = rated.distinct().count()
print('{} movies are rated by users.'.format(rated_count))

9724 movies are rated by users.

In [22]:
#List movies that are not rated by users
unrated=movies_df.join(rated, movies_df.movieId==rated.ratedId, "left_outer")\
.where('ratedId is null')\
.select('movieId','title','genres')
unrated.show()

+-------+--------------------+--------------------+
movieId| title| genres|
+-------+--------------------+--------------------+
 1076|Innocents, The (1...|Drama|Horror|Thri...|
 2939| Niagara (1953)| Drama|Thriller|
 3338|For All Mankind (...| Documentary|
 3456|Color of Paradise...| Drama|
 4194|I Know Where I'm ...| Drama|Romance|War|
 5721| Chosen, The (1981)| Drama|
 6668|Road Home, The (W...| Drama|Romance|
 6849| Scrooge (1970)|Drama|Fantasy|Mus...|
 7020| Proof (1991)|Comedy|Drama|Romance|
 7792|Parallax View, Th...| Thriller|
 8765|This Gun for Hire...|Crime|Film-Noir|T...|
 25855|Roaring Twenties,...|Crime|Drama|Thriller|
 26085|Mutiny on the Bou...|Adventure|Drama|R...|
 30892|In the Realms of ...|Animation|Documen...|
 32160|Twentieth Century...| Comedy|
 32371|Call Northside 77...|Crime|Drama|Film-...|
 34482|Browning Version,...| Drama|
 85565| Chalet Girl (2011)| Comedy|Romance|
+-------+--------------------+--------------------+

In [23]:
unrated_count = unrated.distinct().count()
percent_of_unrated = round(unrated_count/movie_count*100,2)
print('{}% of movies are not rated by users.'.format(percent_of_unrated))

0.18% of movies are not rated by users.

In [24]:
# count of different rating scores
movie_dis = ratings_df.groupBy('movieId').count()
movie_dis.show()

+-------+-----+
movieId|count|
+-------+-----+
 296| 307|
 1090| 63|
 115713| 28|
 3210| 42|
 88140| 32|
 829| 9|
 2088| 18|
 2294| 45|
 4821| 5|
 48738| 20|
 3959| 8|
 89864| 19|
 2136| 14|
 691| 3|
 3606| 4|
 121007| 1|
 6731| 8|
 27317| 6|
 26082| 3|
 100553| 2|
+-------+-----+
only showing top 20 rows

In [25]:
#Visualize the pertentile distribution of movies with regard to rating numbers
fig = plt.figure(figsize=(8,6))
sns.set(style="whitegrid")
sns.distplot(movie_dis.toPandas()['count'], color='green', hist=True)
plt.title('Distribution of Movie Ratings') 
plt.ylabel('percentage')
plt.xlabel('number of ratings per movie')
plt.show()

The movie rating numbers distribution per movie is left-skewed, meaning most of the movies have very small amount of ratings (under around 100).

In [27]:
# count of different rating scores
user_dis = ratings_df.groupBy('userId').count()
user_dis.toPandas().head(20)

,userId,count
0,296,27
1,467,22
2,125,360
3,451,34
4,7,152
5,51,359
6,124,50
7,447,78
8,591,54
9,307,975


In [28]:
#Visualize the pertentile distribution of users with regard to rating numbers
fig = plt.figure(figsize=(8,6))
sns.set(style="whitegrid")
sns.distplot(user_dis.toPandas()['count'], color='salmon', hist=True)
plt.title('Distribution of Movie Ratings') 
plt.ylabel('percentage')
plt.xlabel('number of ratings per movie')
plt.show()

The movie rating numbers per user distribution is left skewed, most users have rating numbers under around 200.

In [30]:
#Average rating of each movie
temp = spark.sql(
'''select movieId as movie_id, avg(rating) as average_rating from ratings
group by movieId''').toPandas()
temp.head(10)

,movie_id,average_rating
0,296,4.197068
1,1090,3.984127
2,115713,3.910714
3,3210,3.476190
4,88140,3.546875
5,829,2.666667
6,2088,2.500000
7,2294,3.244444
8,4821,3.100000
9,48738,3.975000


### Q4: List Movie Genres

In [32]:
movies_df.select('genres').distinct().toPandas()


,genres
0,Comedy|Horror|Thriller
1,Adventure|Sci-Fi|Thriller
2,Action|Adventure|Drama|Fantasy
3,Action|Drama|Horror
4,Action|Animation|Comedy|Sci-Fi
5,Animation|Children|Drama|Musical|Romance
6,Action|Adventure|Drama
7,Adventure|Sci-Fi
8,Documentary|Musical|IMAX
9,Adventure|Children|Fantasy|Sci-Fi|Thriller


In [33]:
#Manipulate the genre colummn
movies_temp=movies_df.where('genres is not null')\
.select('title', F.explode(F.split('genres', '\|'))\
.alias('genres'))
movies_temp.toPandas().head(10)

,title,genres
0,Toy Story (1995),Adventure
1,Toy Story (1995),Animation
2,Toy Story (1995),Children
3,Toy Story (1995),Comedy
4,Toy Story (1995),Fantasy
5,Jumanji (1995),Adventure
6,Jumanji (1995),Children
7,Jumanji (1995),Fantasy
8,Grumpier Old Men (1995),Comedy
9,Grumpier Old Men (1995),Romance


### Q5: Movie for Each Category

In [35]:
movies_pdf = movies_temp.toPandas()
category = pd.DataFrame((movies_pdf.genres).value_counts()).reset_index()
category.columns = ['Category','Number']
category.head(10)

,Category,Number
0,Drama,4361
1,Comedy,3756
2,Thriller,1894
3,Action,1828
4,Romance,1596
5,Adventure,1263
6,Crime,1199
7,Sci-Fi,980
8,Horror,978
9,Fantasy,779


In [36]:
plt.figure(figsize=(60,10))
tmp=sns.set(style="whitegrid")
tmp=sns.barplot(x='Category', y='Number', data=category)
tmp.set_title('Number of movies per category')
tmp.show()

In [37]:
movies_genres=movies_temp.groupBy('genres')\
.agg(F.collect_list("title")\
.alias('titles'))
genre_set = set(movies_genres.toPandas()['genres'])
genre_set

Out[35]: {'(no genres listed)',
 'Action',
 'Adventure',
 'Animation',
 'Children',
 'Comedy',
 'Crime',
 'Documentary',
 'Drama',
 'Fantasy',
 'Film-Noir',
 'Horror',
 'IMAX',
 'Musical',
 'Mystery',
 'Romance',
 'Sci-Fi',
 'Thriller',
 'War',
 'Western'}

## Part2: Spark ALS based approach for training model
We will use an Spark ML to predict the ratings, so let's reload "ratings.csv" using ``sc.textFile`` and then convert it to the form of (user, item, rating) tuples.

In [39]:
ratings_df.show()

+------+-------+------+---------+
userId|movieId|rating|timestamp|
+------+-------+------+---------+
 1| 1| 4.0|964982703|
 1| 3| 4.0|964981247|
 1| 6| 4.0|964982224|
 1| 47| 5.0|964983815|
 1| 50| 5.0|964982931|
 1| 70| 3.0|964982400|
 1| 101| 5.0|964980868|
 1| 110| 4.0|964982176|
 1| 151| 5.0|964984041|
 1| 157| 5.0|964984100|
 1| 163| 5.0|964983650|
 1| 216| 5.0|964981208|
 1| 223| 3.0|964980985|
 1| 231| 5.0|964981179|
 1| 235| 4.0|964980908|
 1| 260| 5.0|964981680|
 1| 296| 3.0|964982967|
 1| 316| 3.0|964982310|
 1| 333| 5.0|964981179|
 1| 349| 4.0|964982563|
+------+-------+------+---------+
only showing top 20 rows

In [40]:
#Convert data types
movie_ratings=ratings_df.drop('timestamp')
movie_ratings=movie_ratings.withColumn('userId',movie_ratings['userId'].cast(IntegerType()))
movie_ratings=movie_ratings.withColumn('movieId',movie_ratings['movieId'].cast(IntegerType()))
movie_ratings=movie_ratings.withColumn('rating',movie_ratings['rating'].cast(FloatType()))

In [41]:
movie_ratings.show()

+------+-------+------+
userId|movieId|rating|
+------+-------+------+
 1| 1| 4.0|
 1| 3| 4.0|
 1| 6| 4.0|
 1| 47| 5.0|
 1| 50| 5.0|
 1| 70| 3.0|
 1| 101| 5.0|
 1| 110| 4.0|
 1| 151| 5.0|
 1| 157| 5.0|
 1| 163| 5.0|
 1| 216| 5.0|
 1| 223| 3.0|
 1| 231| 5.0|
 1| 235| 4.0|
 1| 260| 5.0|
 1| 296| 3.0|
 1| 316| 3.0|
 1| 333| 5.0|
 1| 349| 4.0|
+------+-------+------+
only showing top 20 rows

### ALS Model Selection and Evaluation

With the ALS model, we can use a grid search to find the optimal hyperparameters.

In [43]:
# import package
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS
from pyspark.ml.tuning import CrossValidator,ParamGridBuilder

In [44]:
#Create test and train set
(training,test)=movie_ratings.randomSplit([0.8,0.2])
#Create ALS model
als = ALS(maxIter=5, rank=10, regParam=0.01, userCol="userId", itemCol="movieId", ratingCol="rating",
          coldStartStrategy="drop")
#Tune model using ParamGridBuilder
paramGrid = ParamGridBuilder()\
            .addGrid(als.regParam, [0.03,0.001,0.1])\
            .addGrid(als.maxIter, [3,5,10])\
            .addGrid(als.rank, [5,10,15])\
            .build()
# Define evaluator as RMSE
evaluator = RegressionEvaluator(metricName='rmse', labelCol = 'rating', predictionCol = 'prediction')
# Build Cross validation 
crossval = CrossValidator(estimator=als,
                         estimatorParamMaps=paramGrid,
                         evaluator=evaluator,
                         numFolds=5)
#Fit ALS model to training data
model = als.fit(training)
#Extract best model from the tuning exercise using ParamGridBuilder
cvModel = crossval.fit(training)
predictions = cvModel.transform(training)
rmse = evaluator.evaluate(predictions)
print('Root-mean-squre error = ' + str(rmse))

/databricks/spark/python/pyspark/ml/util.py:791: UserWarning: Can not find mlflow. To enable mlflow logging, install MLflow library from PyPi.
 warnings.warn(_MLflowInstrumentation._NO_MLFLOW_WARNING)
Root-mean-squre error = 0.6379770638913145

### Model testing
And finally, make a prediction and check the testing error.

In [46]:
#Generate predictions and evaluate using RMSE
best_model = cvModel.bestModel
predictions=best_model.transform(test)
rmse = evaluator.evaluate(predictions)

In [47]:
#Print evaluation metrics and model parameters
print ("RMSE = "+str(rmse))
print ("**Best Model**")
print (" Rank:"+str(best_model.rank))
print (" MaxIter:"+str(best_model._java_obj.parent().getMaxIter()))
print (" RegParam:"+str(best_model._java_obj.parent().getRegParam()))

RMSE = 0.8875221067750486
**Best Model**
 Rank:5
 MaxIter:10
 RegParam:0.1

In [48]:
predictions.show()

+------+-------+------+----------+
userId|movieId|rating|prediction|
+------+-------+------+----------+
 597| 471| 2.0| 4.091734|
 603| 471| 4.0| 2.614316|
 411| 471| 4.0| 3.4316437|
 287| 471| 4.5| 2.6037319|
 217| 471| 2.0| 3.4511237|
 312| 471| 4.0| 3.3542545|
 32| 471| 3.0| 3.5989785|
 541| 471| 3.0| 4.2093425|
 218| 471| 4.0| 1.9618757|
 474| 471| 3.0| 3.8781333|
 182| 471| 4.5| 3.0646267|
 57| 471| 3.0| 3.638772|
 448| 471| 4.0| 3.2805827|
 599| 833| 1.5| 1.8755567|
 492| 833| 4.0| 2.0771801|
 307| 833| 1.0| 0.6755262|
 414| 1088| 3.0| 3.2945836|
 188| 1088| 4.0| 3.6272526|
 116| 1088| 4.5| 3.42535|
 216| 1238| 5.0| 3.8883975|
+------+-------+------+----------+
only showing top 20 rows

### Model apply and see the performance

In [50]:
#Fit the best model with the full dataset
alldata=best_model.transform(movie_ratings)
rmse = evaluator.evaluate(alldata)
print ("RMSE = "+str(rmse))

RMSE = 0.6931610901837421

In [51]:
alldata=alldata.withColumn('rounded_predictions', F.round(F.col('prediction')*2)/2)
alldata.registerTempTable("alldata")

In [52]:
#Show the predictions of all movies' ratings
alldata.show()

+------+-------+------+----------+-------------------+
userId|movieId|rating|prediction|rounded_predictions|
+------+-------+------+----------+-------------------+
 191| 148| 5.0| 4.9328084| 5.0|
 91| 471| 1.0| 2.204253| 2.0|
 409| 471| 3.0| 3.188571| 3.0|
 57| 471| 3.0| 3.638772| 3.5|
 597| 471| 2.0| 4.091734| 4.0|
 217| 471| 2.0| 3.4511237| 3.5|
 312| 471| 4.0| 3.3542545| 3.5|
 287| 471| 4.5| 2.6037319| 2.5|
 104| 471| 4.5| 3.8516898| 4.0|
 599| 471| 2.5| 2.779261| 3.0|
 218| 471| 4.0| 1.9618757| 2.0|
 133| 471| 4.0| 3.250437| 3.5|
 436| 471| 3.0| 3.8072963| 4.0|
 372| 471| 3.0| 3.886146| 4.0|
 385| 471| 4.0| 3.2269683| 3.0|
 182| 471| 4.5| 3.0646267| 3.0|
 474| 471| 3.0| 3.8781333| 4.0|
 387| 471| 3.0| 2.7166896| 2.5|
 603| 471| 4.0| 2.614316| 2.5|
 610| 471| 4.0| 3.8403022| 4.0|
+------+-------+------+----------+-------------------+
only showing top 20 rows

In [53]:
# Join the prediction result with the movies dataset
all_movies_rating = alldata.join(movies_df,alldata.movieId == movies_df.movieId, 'left')
all_movies_rating.show()

+------+-------+------+----------+-------------------+-------+--------------------+------+
userId|movieId|rating|prediction|rounded_predictions|movieId| title|genres|
+------+-------+------+----------+-------------------+-------+--------------------+------+
 191| 148| 5.0| 4.9328084| 5.0| 148|Awfully Big Adven...| Drama|
 597| 471| 2.0| 4.091734| 4.0| 471|Hudsucker Proxy, ...|Comedy|
 500| 471| 1.0| 2.3959703| 2.5| 471|Hudsucker Proxy, ...|Comedy|
 555| 471| 3.0| 3.4010515| 3.5| 471|Hudsucker Proxy, ...|Comedy|
 133| 471| 4.0| 3.250437| 3.5| 471|Hudsucker Proxy, ...|Comedy|
 91| 471| 1.0| 2.204253| 2.0| 471|Hudsucker Proxy, ...|Comedy|
 409| 471| 3.0| 3.188571| 3.0| 471|Hudsucker Proxy, ...|Comedy|
 385| 471| 4.0| 3.2269683| 3.0| 471|Hudsucker Proxy, ...|Comedy|
 218| 471| 4.0| 1.9618757| 2.0| 471|Hudsucker Proxy, ...|Comedy|
 474| 471| 3.0| 3.8781333| 4.0| 471|Hudsucker Proxy, ...|Comedy|
 436| 471| 3.0| 3.8072963| 4.0| 471|Hudsucker Proxy, ...|Comedy|
 372| 471| 3.0| 3.886146| 4.0| 471|Hudsucker Proxy, ...|Comedy|
 599| 471| 2.5| 2.779261| 3.0| 471|Hudsucker Proxy, ...|Comedy|
 312| 471| 4.0| 3.3542545| 3.5| 471|Hudsucker Proxy, ...|Comedy|
 287| 471| 4.5| 2.6037319| 2.5| 471|Hudsucker Proxy, ...|Comedy|
 414| 471| 5.0| 3.5849106| 3.5| 471|Hudsucker Proxy, ...|Comedy|
 608| 471| 1.5| 2.3428788| 2.5| 471|Hudsucker Proxy, ...|Comedy|
 373| 471| 5.0| 4.3989425| 4.5| 471|Hudsucker Proxy, ...|Comedy|
 602| 471| 4.0| 3.5783796| 3.5| 471|Hudsucker Proxy, ...|Comedy|
 462| 471| 2.5| 2.2257245| 2.0| 471|Hudsucker Proxy, ...|Comedy|
+------+-------+------+----------+-------------------+-------+--------------------+------+
only showing top 20 rows

In [54]:
#Rank the movie by the movie's total rating counts
movie_rating = ratings_df.groupBy('movieId').count()\
.orderBy('count', ascending = False).toPandas()
movie_rating.head(10)

,movieId,count
0,356,329
1,318,317
2,296,307
3,593,279
4,2571,278
5,260,251
6,480,238
7,110,237
8,589,224
9,527,220


In [55]:
movie_rating.tail(10)

,movieId,count
9714,77233,1
9715,5390,1
9716,85394,1
9717,96114,1
9718,6912,1
9719,2655,1
9720,108090,1
9721,5269,1
9722,27426,1
9723,130840,1


In [56]:
#plot actual and predicted ratings for movie with most number of ratings - take user 356 as an example
movie_top = alldata.where('movieId=356')\
                       .select('rating','rounded_predictions')\
                       .orderBy('rating')
rmse_movie = RegressionEvaluator(metricName='rmse', labelCol='rating', predictionCol='rounded_predictions')
print(rmse_movie.evaluate(movie_top))
movie_top.show()

0.7188305260824444
+------+-------------------+
rating|rounded_predictions|
+------+-------------------+
 0.5| 2.5|
 1.0| 2.5|
 2.0| 3.5|
 2.0| 3.0|
 2.0| 3.5|
 2.0| 3.5|
 2.0| 3.5|
 2.0| 4.0|
 2.0| 3.0|
 2.0| 4.0|
 2.0| 3.5|
 2.5| 2.0|
 2.5| 3.0|
 2.5| 3.0|
 3.0| 4.0|
 3.0| 3.5|
 3.0| 3.5|
 3.0| 4.0|
 3.0| 3.5|
 3.0| 4.0|
+------+-------------------+
only showing top 20 rows

In [57]:
movie_top = movie_top.toPandas()
fig = plt.figure()
plt.plot(movie_top.index, movie_top.rating, 'o', label='rating')
plt.plot(movie_top.index, movie_top.rounded_predictions, '.', label='rounded_predictions')
plt.xlabel('index')
plt.ylabel('ratings')
plt.legend()
plt.title('Actual & predicted ratings for movie with 329 ratings')
plt.show()

In [58]:
rating_rank = ratings_df.groupBy('movieId').count()

In [59]:
#plot actual and predicted ratings for movie with 1 rating
movie_least = alldata.join(rating_rank, rating_rank.movieId==alldata.movieId, 'left')\
.where('count=1').select('rating', 'rounded_predictions').orderBy('rating')
rmse_movie2 = RegressionEvaluator(metricName='rmse', labelCol='rating', predictionCol='rounded_predictions')
print(rmse_movie2.evaluate(movie_least))

0.0

In [60]:
movie_least = movie_least.toPandas()
fig = plt.figure()
plt.plot(movie_least.index, movie_least.rating, 'o', label='rating')
plt.plot(movie_least.index, movie_least.rounded_predictions, '-', label='rounded_predictions')
plt.xlabel('index')
plt.ylabel('ratings')
plt.legend()
plt.title('Actual & predicted ratings for movie with 1 ratings')
plt.show()

In [61]:
#Ranks the user by the user's total rating counts
user_rating = ratings_df.groupBy('userId').count()\
.orderBy('count', ascending = False).toPandas()
user_rating.head(10)

,userId,count
0,414,2698
1,599,2478
2,474,2108
3,448,1864
4,274,1346
5,610,1302
6,68,1260
7,380,1218
8,606,1115
9,288,1055


In [62]:
user_rating.tail(10)

,userId,count
600,53,20
601,194,20
602,595,20
603,576,20
604,406,20
605,431,20
606,278,20
607,320,20
608,257,20
609,569,20


In [63]:
#plot actual and predicted ratings for movie of the most active user - take user 414 as an example
user_top = alldata.where('userId=414')\
                       .select('rating','rounded_predictions')\
                       .orderBy('rating')
rmse_movie = RegressionEvaluator(metricName='rmse', labelCol='rating', predictionCol='rounded_predictions')
print(rmse_movie.evaluate(user_top))
user_top.show()

0.6074413242495237
+------+-------------------+
rating|rounded_predictions|
+------+-------------------+
 0.5| 1.0|
 1.0| 2.5|
 1.0| 3.0|
 1.0| 2.5|
 1.0| 1.5|
 1.0| 1.0|
 1.0| 2.5|
 1.0| 2.0|
 1.0| 2.5|
 1.0| 2.5|
 1.0| 2.0|
 1.0| 1.5|
 1.0| 1.0|
 1.0| 2.5|
 1.0| 2.5|
 1.0| 1.5|
 1.0| 2.5|
 1.0| 3.0|
 1.0| 1.5|
 1.0| 1.5|
+------+-------------------+
only showing top 20 rows

In [64]:
user_top  = user_top.toPandas()
fig = plt.figure()
plt.plot(user_top.index, user_top.rating, 'o', label='rating')
plt.plot(user_top.index, user_top.rounded_predictions, '.', label='rounded_predictions')
plt.xlabel('index')
plt.ylabel('ratings')
plt.legend()
plt.title('Actual & predicted ratings for the most active user')
plt.show()

In [65]:
#plot actual and predicted ratings for movie of the least active user - take user 189 as an example
user_least = alldata.where('userId=189')\
                       .select('rating','rounded_predictions')\
                       .orderBy('rating')
rmse_movie = RegressionEvaluator(metricName='rmse', labelCol='rating', predictionCol='rounded_predictions')
print(rmse_movie.evaluate(user_least))
user_least.show()

0.37080992435478316
+------+-------------------+
rating|rounded_predictions|
+------+-------------------+
 3.5| 4.0|
 3.5| 4.0|
 4.0| 4.0|
 4.0| 4.0|
 4.0| 4.0|
 4.0| 4.0|
 4.0| 4.0|
 4.0| 4.5|
 4.0| 4.0|
 4.0| 4.0|
 4.0| 4.0|
 4.0| 4.0|
 4.0| 4.0|
 4.0| 4.0|
 4.0| 4.0|
 4.5| 4.0|
 4.5| 4.0|
 4.5| 4.0|
 4.5| 4.0|
 5.0| 4.0|
+------+-------------------+

In [66]:
user_least  = user_least .toPandas()
fig = plt.figure()
plt.plot(user_least .index, user_least .rating, 'o', label='rating')
plt.plot(user_least .index, user_least .rounded_predictions, '-', label='rounded_predictions')
plt.xlabel('index')
plt.ylabel('ratings')
plt.legend()
plt.title('Actual & predicted ratings for the least active user')
plt.show()

In [67]:
#Plot the mse of movie ratings by movies of the same rating numbers
alldata_df = alldata.toPandas()
rating_num = alldata_df.groupby(['movieId']).size().reset_index(name='number_of_ratings')
rating_movie = alldata_df.merge(rating_num, on=['movieId'])
rating_movie['MSE']=(rating_movie['rating']-rating_movie['prediction'])**2
rating_movie=rating_movie[['number_of_ratings','MSE']].groupby(by=['number_of_ratings'])['MSE'].mean().reset_index()
rating_movie['Percentile_rank']=rating_movie.number_of_ratings.rank(pct=True)
rating_movie['Percentile_rank']=rating_movie['Percentile_rank'].apply(lambda x: round(x,2))
rating_movie.head(10)


,number_of_ratings,MSE,Percentile_rank
0,1,0.005713,0.01
1,2,0.286676,0.01
2,3,0.354194,0.02
3,4,0.316392,0.02
4,5,0.325650,0.03
5,6,0.343710,0.03
6,7,0.387239,0.04
7,8,0.384564,0.05
8,9,0.351129,0.05
9,10,0.392061,0.06


In [68]:
temp=sns.scatterplot(x='Percentile_rank', y='MSE', data=rating_movie,hue='MSE', legend=False, palette='Blues')
temp.set_title('mean squred error of movies with the same rating numbers')
temp.show()

In [69]:
#Plot the mse of movie ratings by user of the same rating numbers
rating_num = alldata_df.groupby(['userId']).size().reset_index(name='number_of_ratings')
rating_user = alldata_df.merge(rating_num, on=['userId'])
rating_user['MSE']=(rating_user['rating']-rating_user['prediction'])**2
rating_user=rating_user[['number_of_ratings','MSE']].groupby(by=['number_of_ratings'])['MSE'].mean().reset_index()
rating_user['Percentile_rank']=rating_user.number_of_ratings.rank(pct=True)
rating_user['Percentile_rank']=rating_user['Percentile_rank'].apply(lambda x: round(x,2))
rating_user.head(10)

,number_of_ratings,MSE,Percentile_rank
0,20,0.597541,0.00
1,21,0.487525,0.01
2,22,0.644166,0.01
3,23,0.450705,0.02
4,24,0.497283,0.02
5,25,0.613570,0.02
6,26,0.446716,0.03
7,27,0.657273,0.03
8,28,0.443108,0.03
9,29,0.660401,0.04


In [70]:
tmp=sns.scatterplot(x='Percentile_rank', y='MSE', data=rating_user,hue='MSE', legend=False, palette='Blues')
tmp.set_title('mean squred error of users with the same rating numbers')
tmp.show()

## Recommend moive to users with id: 575, 232. 
you can choose some users to recommend the moives

In [72]:
all_movies_rating.where(all_movies_rating.userId==575)\
.select('userId', 'rating', 'title', F.explode(F.split('genres', '\|'))\
.alias('genres'))\
.orderBy('rating', ascending = False).show()
#The user 575 gives the highest ratings to movies of genre Comedy, Dram, Crime and Thriller.

+------+------+--------------------+---------+
userId|rating| title| genres|
+------+------+--------------------+---------+
 575| 5.0|Calendar Girl (1993)| Comedy|
 575| 5.0|Lock, Stock & Two...| Comedy|
 575| 5.0| Pulp Fiction (1994)| Thriller|
 575| 5.0|Lock, Stock & Two...| Thriller|
 575| 5.0|Calendar Girl (1993)| Drama|
 575| 5.0|Lock, Stock & Two...| Crime|
 575| 5.0|William Shakespea...| Comedy|
 575| 5.0|William Shakespea...| Fantasy|
 575| 5.0| Stand by Me (1986)|Adventure|
 575| 5.0| Stand by Me (1986)| Drama|
 575| 5.0| Pulp Fiction (1994)| Comedy|
 575| 5.0| Pulp Fiction (1994)| Crime|
 575| 5.0| Pulp Fiction (1994)| Drama|
 575| 4.0| Matrix, The (1999)| Action|
 575| 4.0|Cookie's Fortune ...| Drama|
 575| 4.0| Matrix, The (1999)| Thriller|
 575| 4.0| Matrix, The (1999)| Sci-Fi|
 575| 4.0| Go (1999)| Crime|
 575| 4.0|Cookie's Fortune ...| Comedy|
 575| 4.0| Go (1999)| Comedy|
+------+------+--------------------+---------+
only showing top 20 rows

In [73]:
all_movies_rating.where(all_movies_rating.userId==232)\
.select('userId', 'rating', 'title', F.explode(F.split('genres', '\|'))\
.alias('genres'))\
.orderBy('rating', ascending = False).show()
#The user 232 gives the highest ratings to movies of genre Adventure, Crime, IMAX, Sci-Fi, Animation, Fantasy, Action, Drama....

+------+------+--------------------+---------+
userId|rating| title| genres|
+------+------+--------------------+---------+
 232| 5.0| Toy Story 3 (2010)|Adventure|
 232| 5.0|Green Mile, The (...| Crime|
 232| 5.0| Toy Story 3 (2010)| IMAX|
 232| 5.0| Pulp Fiction (1994)| Drama|
 232| 5.0|Star Wars: Episod...| Sci-Fi|
 232| 5.0|Dark Knight, The ...| Drama|
 232| 5.0| Toy Story 3 (2010)| Fantasy|
 232| 5.0|We Were Soldiers ...| Action|
 232| 5.0| Pulp Fiction (1994)| Crime|
 232| 5.0|Star Wars: Episod...|Adventure|
 232| 5.0|Star Wars: Episod...| Action|
 232| 5.0|Dark Knight, The ...| Action|
 232| 5.0|Dark Knight, The ...| Crime|
 232| 5.0|Green Mile, The (...| Drama|
 232| 5.0| Toy Story 3 (2010)|Animation|
 232| 5.0|Dark Knight, The ...| IMAX|
 232| 5.0|We Were Soldiers ...| Drama|
 232| 5.0|We Were Soldiers ...| War|
 232| 5.0| Pulp Fiction (1994)| Comedy|
 232| 5.0| Pulp Fiction (1994)| Thriller|
+------+------+--------------------+---------+
only showing top 20 rows

In [74]:
#Generate the top-10 movie recommendations for all users
recommend_for_all=best_model.recommendForAllUsers(10)

In [75]:
#Recommend top-10 movies to user 575
user=575

recommend_for_all=best_model.recommendForAllUsers(10)

recommend_top10=recommend_for_all.where(recommend_for_all.userId==user)\
.select('userId', F.explode('recommendations'))\
.select('userId', F.col('col').movieId.alias('movieId'), F.col('col').rating.alias('rating'))

recommend_top10=recommend_top10.join(movies_df, recommend_top10.movieId==movies_df.movieId, 'left')\
.select('userId', movies_df.movieId, 'title','genres')

recommend_top10.show()

+------+-------+--------------------+--------------------+
userId|movieId| title| genres|
+------+-------+--------------------+--------------------+
 575| 6818|Come and See (Idi...| Drama|War|
 575| 8477| Jetée, La (1962)| Romance|Sci-Fi|
 575| 25771|Andalusian Dog, A...| Fantasy|
 575| 4794| Opera (1987)|Crime|Horror|Mystery|
 575| 112421| Frank (2014)|Comedy|Drama|Mystery|
 575| 7982|Tale of Two Siste...|Drama|Horror|Myst...|
 575| 92494|Dylan Moran: Mons...| Comedy|Documentary|
 575| 102217|Bill Hicks: Revel...| Comedy|
 575| 167746|The Lego Batman M...|Action|Animation|...|
 575| 4617| Let It Ride (1989)| Comedy|
+------+-------+--------------------+--------------------+

In [76]:
#Recommend top-10 movies to user 232
user=232

recommend_for_all=best_model.recommendForAllUsers(10)

recommend_top10=recommend_for_all.where(recommend_for_all.userId==user)\
.select('userId', F.explode('recommendations'))\
.select('userId', F.col('col').movieId.alias('movieId'), F.col('col').rating.alias('rating'))

recommend_top10=recommend_top10.join(movies_df, recommend_top10.movieId==movies_df.movieId, 'left')\
.select('userId', movies_df.movieId, 'title','genres')

recommend_top10.show()

+------+-------+--------------------+--------------------+
userId|movieId| title| genres|
+------+-------+--------------------+--------------------+
 232| 33649| Saving Face (2004)|Comedy|Drama|Romance|
 232| 86237| Connections (1978)| Documentary|
 232| 74226|Dream of Light (a...| Documentary|Drama|
 232| 171495| Cosmos| (no genres listed)|
 232| 179135|Blue Planet II (2...| Documentary|
 232| 184245|De platte jungle ...| Documentary|
 232| 26073|Human Condition I...| Drama|War|
 232| 84273|Zeitgeist: Moving...| Documentary|
 232| 1111|Microcosmos (Micr...| Documentary|
 232| 7071|Woman Under the I...| Drama|
+------+-------+--------------------+--------------------+

## Find the similar moives for moive with id: 463, 471
You can find the similar moives based on the ALS results

In [78]:
movie_factors = best_model.itemFactors
movie_factors = movie_factors.toPandas()
movie_factors.head(10)

,id,features
0,10,"[-0.35784605, -0.16338141, 0.97736216, -1.6528..."
1,20,"[-0.9469013, 0.33816803, 0.5175938, -1.0606815..."
2,30,"[-2.0530734, -0.073496126, 1.0898886, 0.214443..."
3,40,"[-1.3800241, -0.56228113, 0.48996088, -1.92814..."
4,50,"[-1.3005581, -0.14243922, 1.4534355, -1.440206..."
5,60,"[-0.14138323, -0.05522408, 1.0703492, -1.50857..."
6,70,"[-0.8992996, 0.27456158, 0.5469574, -1.5211575..."
7,80,"[-0.8556784, -0.4341444, 1.5677251, -1.0261366..."
8,100,"[-0.59097207, -0.62612736, 0.30054402, -1.5858..."
9,110,"[-0.651647, -0.13331316, 1.293497, -1.7514244,..."


In [79]:

movie_factors.loc[movie_factors['id'] == 471, 'features']



Out[82]: 955 [-0.05414466, -0.74604744, 1.8729309, -1.17062...
Name: features, dtype: object

In [80]:
list(movie_factors['features'][0])

Out[83]: [-0.35784605, -0.16338141, 0.97736216, -1.652817, 0.33503655]

In [81]:
movies_kdf = movies_df.toPandas()
movies_kdf.head(10)

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy
5,6,Heat (1995),Action|Crime|Thriller
6,7,Sabrina (1995),Comedy|Romance
7,8,Tom and Huck (1995),Adventure|Children
8,9,Sudden Death (1995),Action
9,10,GoldenEye (1995),Action|Adventure|Thriller


In [82]:
def similar_movies(features, movieId):
  try:
    target_id_feature = movie_factors.loc[movie_factors['id'] == movieId, 'features'].to_numpy()[0]
  except:
    return 'There is no mivie with id ' + str(movieId)
  similarities=[]
  for feature in movie_factors['features']:
    similarity=float(np.dot(target_id_feature, feature)/(np.linalg.norm(target_id_feature)*np.linalg.norm(feature)))
    similarities.append(similarity)
  ks_similarity = ks.DataFrame({'similarity' : similarities}, index = movie_factors.id.to_numpy()).reset_index().toPandas()
  ks_similarity.columns = ['movieId','similarity']
  ks_similarity['movieId'] = ks_similarity['movieId'].apply(lambda x: str(x))
  movie_similarity = movies_kdf.merge(ks_similarity, on = ['movieId']).sort_values(by='similarity',ascending = False).head(11)
  return movie_similarity
  
#.sort_values(by='similarity',ascending = False).head(11)

In [83]:
#Find the top10 similar movies to movie 463, including itself
similar_movies(movie_factors['features'], 463)



Out[86]: 'There is no mivie with id 463'

In [84]:
#Find the top10 similar movies to movie 471, including itself
similar_movies(movie_factors['features'], 471)

,movieId,title,genres,similarity
396,471,"Hudsucker Proxy, The (1994)",Comedy,1.000000
1092,1465,Rosewood (1997),Action|Drama,0.991152
5469,32289,Ice Princess (2005),Children|Comedy|Drama,0.983565
51,58,"Postman, The (Postino, Il) (1994)",Comedy|Drama|Romance,0.976592
2578,3572,Carnosaur (1993),Horror|Sci-Fi,0.976347
1647,2279,Urban Legend (1998),Horror|Thriller,0.975762
3073,4308,Moulin Rouge (2001),Drama|Musical|Romance,0.974914
746,1013,"Parent Trap, The (1961)",Children|Comedy|Romance,0.973961
624,830,"First Wives Club, The (1996)",Comedy,0.973901
5888,47810,"Wicker Man, The (2006)",Horror|Mystery|Thriller,0.973631


#Report
**Motivation:** I am a big fan of movie and I like rating movies and writing movies on social media apps. While I am using those movie apps, I realized that the more I rated movies, the more accurate the system will recommend good movies to me that fit my taste. I think this is a super fascinating feature and I would love to know how this recommendation really works, so that I could have the ability to build a recommendation system in the future to provide customized user experiences and help the company increase user loyalty.

**Data and Method：** In this work, ALS is used to build a recommendation system based on data from grouplens (Small: 100,000 ratings and 3,600 tag applications applied to 9,000 movies by 600 users. Last updated 9/2018.) One thing to note is that data sparsity is an important challenge within the industry. There can be a lot of users who only have one or two reviews and ratings, and movies which only have a few rating from users. ALS, as a matrix factorization based methodology, is able to increase the scalability and prediction accuracy in this regard.

**Step 1: Data Exploration and Feature Engineering**
<br>
(1) There are 1742 movies in total and 0.18% of them are not rated by any user (610 in total).<br>
(2) The rating number distributions per movie and per user are pretty left-skewed. But there are some special cases where movies and users have an exceptionally large amount ratings. The impact of this will be analyzed in the modeling section.<br>
(3) In terms of movie genres, 'Drama','Comody' and 'Thriller' are the top 3 movie genres in the dataset.<br>
(4) Manipulate the data types, split movie genres into list format for easier usage.<br>

**Step 2: Model Training and Evaluation**
<br>
(1) Build a recommendation model based on historical movie rating data and solve it by alternating least squares model, based on matrix factorization and collaborative filtering method.<br>
(2) Set the cold start parameter in ALS model to 'drop' - in this way, I dropped any rows in the dataset that contain 'NaN' values. The evaluation metric was computed over the non-NaN data so the result would be valid.<br>
(2) Parameters tuned in the model are rank (how many latent features for each user and movie), regParam (regulation paramter), maxIter (max iteration times).<br>
(3) Tuned the model parameters based on grid search and selected the best model with 5-fold cross validation, using rmse (mean root squared errors) as the performance evaluation metric.<br>
(4) The best model has 5 latent features, with rmse = 0.64 on the training dataset and 0.88 on the testing dataset. <br>

**Step 3: Model Testing and Analysis**
<br>
(1) Applied the best model onto the full dataset. The rmse is 0.69.
(2) Explored the model prediction accuracy by visualizing the prediction vs. actual rating of users and movies mith the most/least rating numbers. Then, visualze the mse (mean squared error) of predictions with regard to rating numbers per user/movie.<br>
(3) Conclusion of the above analysis:<br>
Active users give more accurate predictions - this is actually quite reasonable in terms of intuition.
Movie's prediction accuracy is relatively stalbe, and there isn't a downward trend of MSE as the number of rating increases. But intuitively, we would expect that movies with more rating information give more accurate predictions. <br>
If we think about it in this way - very popular movies will be rated by all kinds of users, among which some are active users who have much information on the app, and some are non-active users who have very limited amount of rating record and information on the app. The latter portion can introduce bias into popular movie's predictions, and this may partially explain why movies with more ratings do not offer significantly more accurate predictions.<br>
In this exercise, I don't have a super large dataset, so in order to protect the sufficiency of data for modeling, I did't exlcude any user information. Alternatetively in the next time, I could do a knn clustering on users and movies to decrease the dimension of data before training the model, which would partially solve the sparsity issue. <br>

**Step 4: Model Application on Movie Recommendation**
<br>
(1) Recommended the top 10 movies to certain users, based on user similarity (collaborative filtering) in terms of the 5 latent features of movie defined by the model.<br>
(2) Identified the top 10 movies that are similar to certain movies, based on their cosine similarity calculated from the scores of their 5 lantent features.